In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
print ("Done")

Done


In [5]:
df=pd.read_csv("cleaned_data.csv")
df.head()

Traceback (most recent call last):
  File "C:\Users\Aryan\AppData\Local\Temp\ipykernel_20464\2682650136.py", line 500, in _repr_dw_
    return self._gen_json()
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Aryan\AppData\Local\Temp\ipykernel_20464\2682650136.py", line 508, in _gen_json
    return api["pandas_transport"]["get_df_payload"](tmp_vars[self.id]["converted"], max_num_rows_to_preload, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Aryan\AppData\Local\Temp\ipykernel_20464\2682650136.py", line 248, in get_df_payload
    return get_df_payload_v0(df, row_limit, **extra_props)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Aryan\AppData\Local\Temp\ipykernel_20464\2682650136.py", line 240, in get_df_payload_v0
    return pd_dumps({
           ^^^^^^^^^^
  File "C:\Users\Aryan\AppData\Local\Temp\ipykernel_20464\2682650136.py", line 217, in pd_dumps
    return

,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df.drop(['Surname'],axis=1,inplace=True)
print("Done")


Done


In [5]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
scaler = StandardScaler()
numerical_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary','HasCrCard','IsActiveMember']
categorical_features = ['Geography', 'Gender',]

numerical_pipeline=Pipeline(steps=[('scaler',scaler)])
categorical_pipeline=Pipeline(steps=[('encoder',OneHotEncoder(handle_unknown='ignore'))])

preprocessor=ColumnTransformer(transformers=[('num',numerical_pipeline,numerical_features),('cat',categorical_pipeline,categorical_features)],remainder='passthrough')
print("preprocessing completed")

preprocessing completed


In [8]:
X= df.drop('Exited',axis=1)
y= df['Exited']

x_train_full, x_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
x_train_full, x_valid, y_train_full, y_valid = train_test_split(x_train_full, y_train_full, test_size=0.2, random_state=42)

print("Done")

Done


In [9]:
preprocessor.fit(x_train_full) # this fits the preprocessor on the training data --> the learning phase of the preprocessor

x_train_preprocessed = preprocessor.transform(x_train_full) # applying the learned transformations to the training data
x_valid_preprocessed = preprocessor.transform(x_valid) # applying the learned transformations to the validation data
x_test_preprocessed = preprocessor.transform(x_test) # applying the learned transformations to the test data

print("Data Preprocessing Completed")

Data Preprocessing Completed


In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=x_train_preprocessed.shape[1:]),
    tf.keras.layers.Dense(11, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)

model.fit(x_train_preprocessed, y_train_full, epochs=100, batch_size=32, validation_data=(x_valid_preprocessed, y_valid),callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)], class_weight={0:1.2, 1:2.0})

print("Model Training Completed")

print(model.evaluate(x_test_preprocessed, y_test))

print("Model classification report:")

y_pred_probs = model.predict(x_test_preprocessed)

from sklearn.metrics import classification_report

y_pred = (y_pred_probs > 0.5).astype("int32")

print(classification_report(y_test, y_pred))

model.save('churn_model.h5')

import joblib
joblib.dump(preprocessor, 'preprocessor.pkl')

C:\Users\Aryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 11)             │           154 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 50)             │           600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 50)             │         2,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,355 (13.11 KB)

 Trainable params: 3,355 (13.11 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7850 - loss: 0.7745 - precision: 0.4374 - recall: 0.1564 - val_accuracy: 0.7987 - val_loss: 0.4435 - val_precision: 0.5126 - val_recall: 0.3119
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8087 - loss: 0.6901 - precision: 0.5467 - recall: 0.4131 - val_accuracy: 0.8200 - val_loss: 0.4005 - val_precision: 0.5789 - val_recall: 0.4373
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8305 - loss: 0.6310 - precision: 0.6158 - recall: 0.4685 - val_accuracy: 0.8325 - val_loss: 0.3921 - val_precision: 0.5937 - val_recall: 0.5719
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8381 - loss: 0.5993 - precision: 0.6319 - recall: 0.5110 - val_accuracy: 0.8594 - val_loss: 0.3545 - val_precision: 0.7339 - val_recall: 0.4893
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8438 - loss: 0.5846 - precision: 0.6491 - recall: 0.5239 - val_accuracy: 0.8475 - val_los

              precision    recall  f1-score   support

           0       0.89      0.94      0.91      1607
           1       0.68      0.54      0.60       393

    accuracy                           0.86      2000
   macro avg       0.78      0.74      0.76      2000
weighted avg       0.85      0.86      0.85      2000



['preprocessor.pkl']

In [15]:
wide_features=['Balance','NumOfProducts']
deep_features=['CreditScore', 'Age', 'Tenure', 'EstimatedSalary','HasCrCard','IsActiveMember','Geography', 'Gender']

wide_pipeline=Pipeline(steps=[('scaler', scaler)])
deep_pipeline=Pipeline(steps=[('scaler', scaler),('encoder', OneHotEncoder(handle_unknown='ignore'))])


deep_numerical_features = ['CreditScore', 'Age', 'Tenure', 'EstimatedSalary', 'HasCrCard', 'IsActiveMember']
deep_categorical_features = ['Geography', 'Gender']


wide_preprocessor=ColumnTransformer(transformers=[('wide',wide_pipeline,wide_features)],remainder='drop')
deep_preprocessor=ColumnTransformer(transformers=[('num',deep_pipeline,deep_numerical_features),('cat',OneHotEncoder(handle_unknown='ignore'),deep_categorical_features)],remainder='drop')

wide_preprocessor.fit(x_train_full)
deep_preprocessor.fit(x_train_full)

x_train_wide=wide_preprocessor.transform(x_train_full)
x_train_deep=deep_preprocessor.transform(x_train_full)

X_valid_wide = wide_preprocessor.transform(x_valid)
X_valid_deep = deep_preprocessor.transform(x_valid)

X_test_wide = wide_preprocessor.transform(x_test)
X_test_deep = deep_preprocessor.transform(x_test)

print("Data Preprocessing Completed")


Data Preprocessing Completed


In [17]:
input_wide=tf.keras.Input(shape=x_train_wide.shape[1:])
input_deep=tf.keras.Input(shape=x_train_deep.shape[1:])

hidden_1=tf.keras.layers.Dense(64,activation='relu')(input_deep)
hidden_2=tf.keras.layers.Dense(32,activation='relu')(hidden_1)
hidden_3=tf.keras.layers.Dense(16,activation='relu')(hidden_2)
concat_layer=tf.keras.layers.concatenate([input_wide,hidden_3])
output_layer=tf.keras.layers.Dense(1,activation='sigmoid')(concat_layer)

model=tf.keras.Model(inputs=[input_wide,input_deep],outputs=output_layer)

print("Model Summary:")
model.summary()

Model Summary:


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 6934)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64)        │    443,840 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      2,080 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_3       │ (None, 2)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 16)        │        528 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 18)        │          0 │ input_layer_3[0]… │
│ (Concatenate)       │                   │            │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │         19 │ concatenate_1[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 446,467 (1.70 MB)

 Trainable params: 446,467 (1.70 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
from sklearn.metrics import classification_report

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit([x_train_wide, x_train_deep], y_train_full, epochs=200, batch_size=32, validation_data=([X_valid_wide, X_valid_deep], y_valid), callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)], class_weight={0:1.0,1:2.0})
y_pred_probs = model.predict([X_test_wide, X_test_deep])
if y_pred_probs is not None:
    y_pred = (y_pred_probs > 0.5).astype("int32")
    print("Model classification report:")
    print(classification_report(y_test, y_pred))

Epoch 1/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9728 - loss: 0.1263 - val_accuracy: 0.6919 - val_loss: 0.7132
Epoch 2/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9981 - loss: 0.0161 - val_accuracy: 0.7419 - val_loss: 0.6657
Epoch 3/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 0.7269 - val_loss: 0.6977
Epoch 4/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 1.0000 - loss: 8.4039e-04 - val_accuracy: 0.7144 - val_loss: 0.7374
Epoch 5/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 1.0000 - loss: 4.9497e-04 - val_accuracy: 0.7206 - val_loss: 0.7430
Epoch 6/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 1.0000 - loss: 3.2699e-04 - val_accuracy: 0.7206 - val_loss: 0.7613
Epoch 7/200
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 1.0000 - loss: 2.2965e-04 - val_accuracy: 0.7269 - val_loss: 0.7689
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Model classification report:
        